In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import keras
import matplotlib.pyplot as plt
import xlsxwriter
def savexlsx(data,name):
    workbook=xlsxwriter.Workbook('{}.xlsx'.format(name))
    worksheet=workbook.add_worksheet()
    hang = data.shape[0]
    lie = data.shape[1]
    for col in range(int(hang)):
        for row in range(int(lie)):
            worksheet.write(col,row,data[col,row])
    workbook.close()

In [ ]:
Split_or_not = True


# Training

In [ ]:
dataset = pd.read_excel('data.xlsx', header = None)

In [ ]:
orignal_data = np.array(dataset)

In [ ]:
orignal_data.shape

In [ ]:
data = orignal_data[:,:473]
label = orignal_data[:,473:]

In [ ]:
if Split_or_not  == True:
    train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.3)
    train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.1)
else:
    train_x = pd.read_excel('train_x.xlsx', header=None).values
    valid_x = pd.read_excel('valid_x.xlsx', header=None).values
    train_y = pd.read_excel('train_y.xlsx', header=None).values
    valid_y = pd.read_excel('valid_y.xlsx', header=None).values
    test_x = pd.read_excel('test_x.xlsx', header=None).values
    test_y = pd.read_excel('test_y.xlsx', header=None).values

In [ ]:
train_y1 = train_y[:,0]
train_y2 = keras.utils.np_utils.to_categorical(train_y[:,1])
valid_y1 = valid_y[:,0]
valid_y2 = keras.utils.np_utils.to_categorical(valid_y[:,1])
test_y1 = test_y[:,0]
test_y2 = keras.utils.to_categorical(test_y[:,1])

In [ ]:
savexlsx(train_x,'train_x-1')
savexlsx(valid_x,'valid_x-1')
savexlsx(test_x,'test_x-1')
savexlsx(train_y,'train_y-1')
savexlsx(valid_y,'valid_y-1')
savexlsx(test_y,'test_y-1')

In [ ]:
from keras import regularizers
from keras.layers import Reshape,Add,Multiply,merge,Input,Dense, Activation,Dense,Dropout,Flatten,BatchNormalization,Conv1D
from keras.layers import MaxPooling1D,AveragePooling1D,LeakyReLU,GlobalAveragePooling1D
from keras.models import Sequential,Model
from keras.optimizers import Adam,SGD,Adamax
from keras.callbacks import ModelCheckpoint
from keras import losses
from keras.models import load_model

In [ ]:
train_x = train_x.reshape(train_x.shape[0],train_x.shape[1],1)
valid_x = valid_x.reshape(valid_x.shape[0],valid_x.shape[1],1)

In [ ]:
def Res_block(X,filters):
    
    F1,F2 = filters
 
    X_shortcut = X
 
    X = Conv1D(filters = F1, kernel_size = 1, strides = 1,kernel_initializer='random_uniform',bias_initializer='zeros')(X)
    X = Dropout(0.5)(X)
    X = Activation('relu')(X)
    X = Conv1D(filters = F2, kernel_size = 3, strides = 1, padding = 'same',kernel_initializer='random_uniform',bias_initializer='zeros')(X)
    X_shortcut = Conv1D(F2, kernel_size = 1, strides = 1, kernel_initializer='random_uniform',bias_initializer='zeros')(X_shortcut)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
 
    return X

In [ ]:
def ResNet_new(input_shape = (473,), classes1 = 1, classes2 = 2):
 
    X_input = Input(input_shape)
    X1 = Dense(128, activation = 'relu', name = 'base_1',kernel_initializer='random_uniform',bias_initializer='zeros')(X_input)
    X1 = Dense(473, name = 'base_2', kernel_initializer='random_uniform',bias_initializer='zeros',)(X1)
    X = Multiply()([X1, X_input])

    
    X = Reshape((473, 1))(X)
    X = Res_block(X,filters=[32,16])
 
    X = AveragePooling1D(3, name = 'avg_pool')(X)
    X = Flatten()(X)
    X = Dense(128, activation = 'relu',kernel_initializer='random_uniform',bias_initializer='zeros')(X)
    X = Dropout(0.5)(X)
    X = Dense(64, activation = 'relu',kernel_initializer='random_uniform',bias_initializer='zeros')(X)
    X = Dense(classes1, activation = 'sigmoid',name='x1',kernel_initializer='random_uniform',bias_initializer='zeros')(X)


    X_c = Dense(256, activation = 'relu',name='x2_1',kernel_initializer='random_uniform',bias_initializer='zeros')(X_input)
    X_c = Dense(128, activation = 'relu',name='x2_2',kernel_initializer='random_uniform',bias_initializer='zeros')(X_c)
    X_c = Dense(classes2, activation = 'softmax',name='x2_3',kernel_initializer='random_uniform',bias_initializer='zeros')(X_c)
    
    model = Model(inputs = X_input, outputs = [X,X_c] , name = 'LPNet')
    
    return model

In [ ]:
model = ResNet_new()

In [ ]:
for layers in model.layers:
    name = str(layers.name)
    if name.startswith("x2"):
        layers.trainable=True
    else:
        layers.trainable=False 

In [ ]:
model.compile(loss={'x1':'mean_squared_error','x2_3':'categorical_crossentropy'}, loss_weights={'x1':0,'x2_3':1},optimizer=Adamax(lr=0.0001), metrics=['accuracy']) 

In [ ]:
model_names1 = 'model_cls_epoch:{epoch:02d}-val_acc:{val_x2_4_acc:.4f}.hdf5'
save_model1 = ModelCheckpoint(filepath= model_names1, monitor='val_x2_4_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
record_h=model.fit(train_x, {'x1':train_y1,'x2_4':train_y2}, epochs=6000,batch_size=16, validation_data=(valid_x, {'x1':valid_y1,'x2_4':valid_y2}),callbacks=[save_model1])

In [ ]:
record_h1 = record_h

In [ ]:
plt.plot(record_h1.history['loss'])
plt.plot(record_h1.history['val_loss'])
plt.plot(record_h1.history['x2_4_acc'])
plt.plot(record_h1.history['val_x2_4_acc'])
plt.title("cls model loss and acc")
plt.ylabel("loss and acc") 
plt.xlabel("epoch")
plt.legend(["train_loss","test_loss","train_acc","test_acc"],loc="upper right")
plt.savefig("cls.jpg",dpi=600)
plt.show()

In [ ]:
model1 = load_model('model_cls_epoch:xxxx-val_acc:xxxxx.hdf5')

In [ ]:
y1 = model1.predict(train_x)
y2 = model1.predict(valid_x)
y3 = model1.predict(test_x)

In [ ]:
y11 = np.argmax(y1[1],1).reshape(-1,1)
y12 = np.argmax(y2[1],1).reshape(-1,1)
y13 = np.argmax(y2[1],1).reshape(-1,1)
y14 = y1[0].reshape(-1,1)
y15 = y2[0].reshape(-1,1)
y16 = y3[0].reshape(-1,1)

In [ ]:
yy1 = np.concatenate((y14,y11),axis=1)
yy2 = np.concatenate((y15,y12),axis=1)
yy3 = np.concatenate((y16,y13),axis=1)

In [ ]:
savexlsx(yy1.reshape(-1,2),'train_cls_pre_y')
savexlsx(yy2.reshape(-1,2),'valid_cls_pre_y')
savexlsx(yy3.reshape(-1,2),'test_cls_pre_y')

In [ ]:
for layers in model1.layers:
    name = str(layers.name)
    if name.startswith("x2"):
        layers.trainable=False
    else:
        layers.trainable=True 

In [ ]:
model_names2 = 'model_pro_epoch:{epoch:02d}-val_x1_loss:{val_x1_loss:.4f}.hdf5'
save_model = ModelCheckpoint(filepath= model_names2, monitor='val_x1_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
model1.compile(loss={'x1':'mean_squared_error','x2_4':'categorical_crossentropy'}, loss_weights={'x1':1,'x2_4':0},optimizer=Adamax(lr=0.0001), metrics=['accuracy'])  

In [ ]:
record_h2=model1.fit(train_x, {'x1':train_y1,'x2_4':train_y2}, epochs=6000,batch_size=32, validation_data=(valid_x, {'x1':valid_y1,'x2_4':valid_y2}),callbacks=[save_model])

In [ ]:
plt.plot(record_h2.history['x1_loss'])
plt.plot(record_h2.history['val_x1_loss'])
plt.title("reg model loss and acc")
plt.ylabel("loss and acc") 
plt.xlabel("epoch")
plt.legend(["train_loss","test_loss"],loc="upper right")
plt.savefig("reg.jpg",dpi=600)
plt.show()

In [ ]:
model2 = load_model('model_pro_epoch:xxxx-val_x1_loss:xxxx.hdf5')

In [ ]:
y1 = model1.predict(train_x)
y2 = model1.predict(valid_x)
y1 = y1[0]
y2 = y2[0]

In [ ]:
savexlsx(y1.reshape(-1,1),'train_reg_pre_y')
savexlsx(y2.reshape(-1,1),'test_reg_pre_y')

In [ ]:
loss = record_h1.history['loss']
val_loss = record_h1.history['val_loss']
acc = record_h1.history['x2_4_acc']
val_acc = record_h1.history['val_x2_4_acc']
log_data = np.stack((loss, val_loss, acc,val_acc), axis=1)
savexlsx(log_data,'log1')

# Prediction

In [ ]:
model_p = load_model('model_epoch:487-val_loss:2.4914.hdf5')

In [ ]:
y = model_p.predict(train_x)

In [ ]:
savexlsx(y,'train_pre_y')

In [ ]:
y2 = model_p.predict(valid_x)

In [ ]:
savexlsx(y2,'valid_pre_y')

In [ ]:
y3 = model_p.predict(test_x)
savexlsx(y3,'test_pre_y')

In [ ]:
layer_name = 'base_4'
intermediate_layer_model = Model(inputs=model1.input,outputs=model1.get_layer(layer_name).output)#创建的新模型
output1 = intermediate_layer_model.predict(valid_x)
output2 = intermediate_layer_model.predict(train_x)
output3 = intermediate_layer_model.predict(test_x)

In [ ]:
savexlsx(output1,'valid_attention_value')
savexlsx(output2,'train_attention_value')
savexlsx(output3,'test_attention_value')

In [ ]:
layer_name = 'multiply_2'
intermediate_layer_model = Model(inputs=model1.input,outputs=model1.get_layer(layer_name).output)#创建的新模型
output4 = intermediate_layer_model.predict(valid_x)
output5 = intermediate_layer_model.predict(train_x)
output6 = intermediate_layer_model.predict(test_x)

In [ ]:
savexlsx(output4,'test_attention_out')
savexlsx(output5,'train_attention_out')
savexlsx(output6,'test_attention_out')